<a href="https://colab.research.google.com/github/sriramnurani1995/Data-Engineering-Class-Activities/blob/main/Activity2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

# Function to get latitude and longitude
def get_lat_lon(city, state, country, api_key):
    try:
        # Combine user input into a location query
        location_query = f"{city},{state},{country}"
        # OpenWeatherMap geocoding API endpoint
        geocoding_url = f"http://api.openweathermap.org/geo/1.0/direct?q={location_query}&limit=1&appid={api_key}"

        # Make a request to the geocoding API
        response = requests.get(geocoding_url)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code

        # Assuming the first result is the most relevant one
        geocoding_data = response.json()[0]

        # Extract latitude and longitude
        lat = geocoding_data['lat']
        lon = geocoding_data['lon']

        return lat, lon

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # HTTP error
    except Exception as err:
        print(f"Other error occurred: {err}")  # Other errors
    return None, None

# Ask the user for input
city_name = input("Please enter the city name: ")
state_code = input("Please enter the state code: ")
country_code = input("Please enter the country code: ")

# Your OpenWeatherMap API key (You should keep this private and not expose it in shared code)
api_key = '01b1c5ce2b743eb27f35ebbd3b53f96f'

# Call the function to get the latitude and longitude
latitude, longitude = get_lat_lon(city_name, state_code, country_code, api_key)

if latitude is not None and longitude is not None:
    print(f"The latitude and longitude for {city_name}, {state_code}, {country_code} are {latitude}, {longitude}.")
else:
    print("Could not get the geographical coordinates.")


Please enter the city name: PORTLAND
Please enter the state code: OR
Please enter the country code: US
The latitude and longitude for PORTLAND, OR, US are 45.5202471, -122.674194.


In [29]:
import requests

def get_current_weather(lat, lon, api_key):
    try:
        weather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"

        # Make a request to the API
        response = requests.get(weather_url)
        response.raise_for_status()  # Will raise an HTTPError for bad requests

        # Parse the JSON response
        weather_data = response.json()

        # Rain condition codes indicating different types of rain as per OpenWeatherMap documentation
        rain_codes = [200, 201, 202, 230, 231, 232, 300, 301, 302, 310, 311, 312, 313, 314, 321, 500, 501, 502, 503, 504, 511, 520, 521, 522, 531]

        # Iterate through the weather conditions and check for rain codes
        for condition in weather_data['weather']:
            if condition['id'] in rain_codes:
                return True  # It's raining

        return False  # It's not raining

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # HTTP error
    except Exception as err:
        print(f"Other error occurred: {err}")  # Other errors
    return False


# Your OpenWeatherMap API key
api_key = '01b1c5ce2b743eb27f35ebbd3b53f96f'

# Check if it is currently raining
is_raining = get_current_weather(latitude, longitude, api_key)

print(f"Is it raining in Portland, OR right now? {'Yes' if is_raining else 'No'}")


Is it raining in Portland, OR right now? No


In [28]:
import requests
from datetime import datetime, timedelta

def get_next_class_datetime(now, class_days, class_start_time):
    # Calculate the next class datetime based on the current datetime and class schedule
    today_day_number = now.weekday()
    days_ahead = sorted((class_day - today_day_number) % 7 for class_day in class_days)
    # If today is a class day and it hasn't started yet, consider today as the next class date
    if today_day_number in class_days and now.time() < class_start_time:
        next_class_date = now.date()
    else:
        next_class_date = now.date() + timedelta(days=days_ahead[0])
    return datetime.combine(next_class_date, class_start_time)

def get_forecast_entry_for_next_class(lat, lon, api_key, next_class_datetime, class_duration_hours):
    # OpenWeatherMap 5-day forecast API endpoint
    forecast_url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}"

    # Make a request to the API
    response = requests.get(forecast_url)
    response.raise_for_status()  # Will raise an HTTPError for bad requests

    # Parse the JSON response
    forecast_data = response.json()
    # Get the list of forecasts
    forecasts = forecast_data['list']

    # Iterate over the forecasts to find the one for the next class period
    for forecast in forecasts:
        forecast_time = datetime.utcfromtimestamp(forecast['dt'])
        # print("Forecast Time :")
        # print(forecast_time)
        # print("")
        # print("Next Class Time :")
        # print(next_class_datetime)
        # Check if the forecast is within the next class period
        if forecast_time <= next_class_datetime < forecast_time + timedelta(hours=3):
            return forecast


    return None  # If no forecast matches the class period, return None

def check_for_rain(forecast_entry):
    if forecast_entry:  # Ensure there is a forecast entry to check
        # Rain condition codes indicating different types of rain
        rain_codes = [200, 201, 202, 230, 231, 232, 300, 301, 302, 310, 311, 312, 313, 314, 321, 500, 501, 502, 503, 504, 511, 520, 521, 522, 531]
        # Check the weather condition codes in the forecast entry
        weather_conditions = forecast_entry['weather']
        for condition in weather_conditions:
            if condition['id'] in rain_codes:
                return True  # Rain is forecasted
    return False  # No rain forecasted or no forecast entry provided

# Class days, start time, and duration (in hours)
class_days = [1, 3]  # Tuesday and Thursday
class_start_time = datetime.strptime("18:30", "%H:%M").time()
class_duration_hours = 2  # Duration of the class in hours

# Determine the datetime of the next class
now = datetime.now()
next_class_datetime = get_next_class_datetime(now, class_days, class_start_time)

# Find the forecast entry for the next class period
forecast_entry = get_forecast_entry_for_next_class(latitude, longitude, api_key, next_class_datetime, class_duration_hours)

# Check if it will rain during the next class
is_raining_next_class = check_for_rain(forecast_entry)

# Print the result
print(f"Will it rain during the next class? {'Yes' if is_raining_next_class else 'No'}")


Will it rain during the next class? Yes


# QUESTION 1 : Is it raining in Portland, OR right now?

No It is not Raining

# QUESTION 2 : Will it rain during the next class?
Yes It will rain during the next Class